In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
# Import system helpers
import os
import pathlib
from os import listdir
from os.path import isfile, join

# Import TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import helper libraries
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import IPython.display as display
from PIL import Image

# Import pipeline
import format_data
import write_tf2record

In [3]:
car_makes, cars_train_labels, cars_train_annotations, cars_test_annotations = format_data.devkit()


In [0]:
# def _bytes_feature(value):
#   """Returns a bytes_list from a string / byte."""
#   if isinstance(value, type(tf.constant(0))):
#     value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
#   return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# def _float_feature(value):
#   """Returns a float_list from a float / double."""
#   return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

# def _int64_feature(value):
#   """Returns an int64_list from a bool / enum / int / uint."""
#   return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


In [0]:
# # This is an example, just using the cat image.
# path = '/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy/'
# img = '00001.jpg'
# image_string = open(path+img, 'rb').read()
# label = cars_train_labels[img]

# # Create a dictionary with features that may be relevant.
# def image_example(image_string, label):
#   image_shape = tf.image.decode_jpeg(image_string).shape

#   feature = {
#       'height': _int64_feature(image_shape[0]),
#       'width': _int64_feature(image_shape[1]),
#       'depth': _int64_feature(image_shape[2]),
#       'label': _int64_feature(label),
#       'image_raw': _bytes_feature(image_string),
#   }

#   return tf.train.Example(features=tf.train.Features(feature=feature))

# #for line in str(image_example(image_string, label)).split('\n')[:15]:
# #  print(line)

In [0]:
# record_file = 'images.tfrecords'
# with tf.io.TFRecordWriter(record_file) as writer:
#   for filename, label in cars_train_labels.items():
#     image_string = open(filfilename, 'rb').read()
#     tf_example = image_example(image_string, label)
#     writer.write(tf_example.SerializeToString())

In [0]:
# !du -sh {record_file}

In [0]:
# raw_image_dataset = tf.data.TFRecordDataset('images.tfrecords')

# # Create a dictionary describing the features.
# image_feature_description = {
#     'height': tf.io.FixedLenFeature([], tf.int64),
#     'width': tf.io.FixedLenFeature([], tf.int64),
#     'depth': tf.io.FixedLenFeature([], tf.int64),
#     'label': tf.io.FixedLenFeature([], tf.int64),
#     'image_raw': tf.io.FixedLenFeature([], tf.string),
# }

# def _parse_image_function(example_proto):
#   # Parse the input tf.Example proto using the dictionary above.
#   return tf.io.parse_single_example(example_proto, image_feature_description)

# parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
# parsed_image_dataset

In [4]:
parsed_image_dataset = write_tf2record.read_record('/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy/')

In [5]:
labels = [image_features['label'].numpy() for image_features in parsed_image_dataset]
images = np.array([image_features['image_raw'].numpy() for image_features in parsed_image_dataset])

In [10]:
%%timeit
tf.image.resize(tf.image.convert_image_dtype(tf.image.decode_jpeg(images[0], channels=3), tf.float32), [224, 224])
#images = np.array([tf.image.resize(tf.image.decode_jpeg(img, channels=3), [224, 224]) for img in images])

10.9 ms ± 410 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
images_decoded = [format_data.decode_img(img) for img in images[:1000]]

In [24]:
labels = np.array(labels)

In [4]:
# image_batch, label_batch = format_data.get_batch('/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy 2/*')

In [5]:
# x = label_batch[0]
# car_makes.index[x]

In [10]:
# YOU SHOULD ONLY CALL THIS ONCE PER DIRECTORY
# format_data.label_crop('/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy 2/')

In [97]:
# def get_label(file_path):
#     # convert the path to a list of path components
#     parts = tf.strings.split(tf.strings.split(file_path, '/')[-1], '_')[0]
#     # The last is the image name
#     return parts==np.array([str(label) for label in cars_train_labels.values()])

# label = get_label('/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy 2/196_07537.jpg')

In [9]:
# # Crop images based on bounding box

# #copy_path = '/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy 2/'
# def crop(copy_path): 
#     for filename in os.listdir(copy_path):
#         img = Image.open(copy_path + filename)
#         bb_x1 = int(cars_train_annotations[cars_train_annotations['img_name'] == filename]['bb_x1'])
#         bb_x2 = int(cars_train_annotations[cars_train_annotations['img_name'] == filename]['bb_x2'])
#         bb_y1 = int(cars_train_annotations[cars_train_annotations['img_name'] == filename]['bb_y1'])
#         bb_y2 = int(cars_train_annotations[cars_train_annotations['img_name'] == filename]['bb_y2'])
#         img2 = img.crop((bb_x1, bb_x2, bb_y1, bb_y2))
#         img2.save(copy_path + filename)

In [8]:
# #crop(copy_path='/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy 2/')
# img = Image.open('/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy 2/00004.jpg')
# width, height = img.size
# bb_x1 = int(cars_train_annotations[cars_train_annotations['img_name'] == '00004.jpg']['bb_x1'])
# bb_x2 = int(cars_train_annotations[cars_train_annotations['img_name'] == '00004.jpg']['bb_x2'])
# bb_y1 = int(cars_train_annotations[cars_train_annotations['img_name'] == '00004.jpg']['bb_y1'])
# bb_y2 = int(cars_train_annotations[cars_train_annotations['img_name'] == '00004.jpg']['bb_y2'])
# img2 = img.crop((bb_x1, bb_x2, bb_y1, bb_y2))

# #print(width, height, bb_x1, bb_x2, bb_y1, bb_y2)
# img2.save(copy_path + filename)
# #img2 = Image.open('/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy 2/a_00004.jpg')
# img2.show()

In [71]:
# # Convert .mat to Python dict with Scipy.io

# sc_devkit = '/Users/katerina/Workspace/visual_census/data/devkit'
# cars_meta = scipy.io.loadmat(os.path.join(sc_devkit, 'cars_meta.mat'))['class_names']
# cars_train_annos = scipy.io.loadmat(os.path.join(sc_devkit, 'cars_train_annos.mat'))['annotations']

In [72]:
# # Extract Car Metadata from dictionary to an array

# car_makes = []
# for idx, vehicle in enumerate(cars_meta[0]):
#     car_makes.append([vehicle[0],vehicle[0].split(' ')[0], vehicle[0].split(' ')[-1]])
# car_makes = pd.DataFrame(car_makes, columns=['full_label', 'mnfr', 'year'])
# car_make_nums = np.array(car_makes.index)

In [73]:
# # Extract Car Labels from dictionary to a list
# # List used because both int (label) and str (img id) required

# cars_train_labels = {}
# cars_train_annotations = []
# for idx, anno in enumerate(cars_train_annos[0]):
#     cars_train_labels[anno[5][0]] = anno[4][0][0]
#     cars_train_annotations.append([anno[0][0][0], anno[1][0][0], anno[2][0][0], anno[3][0][0], anno[4][0][0], anno[5][0]])
# cars_train_annotations = pd.DataFrame(cars_train_annotations, columns=['bb0', 'bb1', 'bb2', 'bb3', 'label', 'img_name'])

In [74]:
# sc_cars_train = '/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy'
# sc_cars_test = '/Users/katerina/Workspace/visual_census/data/training_data/cars_test'
# train_data_dir = pathlib.Path(sc_cars_train)
# test_data_dir = pathlib.Path(sc_cars_test)

In [75]:
# train_image_count = len(list(train_data_dir.glob('*.jpg')))
# test_image_count = len(list(test_data_dir.glob('*.jpg')))

In [76]:
# train_image_names = np.array([item.name for item in train_data_dir.glob('*')])
# test_image_names = np.array([item.name for item in test_data_dir.glob('*')])

In [77]:
# list_ds = tf.data.Dataset.list_files(str('/Users/katerina/Workspace/visual_census/data/training_data/cars_train copy/*'))
# list_ds_test = tf.data.Dataset.list_files(str('/Users/katerina/Workspace/visual_census/data/training_data/cars_test/*'))

In [78]:
# BATCH_SIZE = 32
# IMG_HEIGHT = 224
# IMG_WIDTH = 224
# STEPS_PER_EPOCH = np.ceil(train_image_count/BATCH_SIZE)

In [79]:
# def get_label(file_path):
#   # convert the path to a list of path components
#   parts = tf.strings.split(tf.strings.split(file_path, '_')[-1], '.')[0]
#   # The last is the image name
#   return parts==np.array([str(idx) for idx in car_makes.index])

In [80]:
# def decode_img(img):
#   # convert the compressed string to a 3D uint8 tensor
#   img = tf.image.decode_jpeg(img, channels=3)
#   # Use `convert_image_dtype` to convert to floats in the [0,1] range.
#   img = tf.image.convert_image_dtype(img, tf.float32)
#   # resize the image to the desired size.
#   return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [81]:
# def process_path(file_path):
#   label = get_label(file_path)
#   # load the raw data from the file as a string
#   img = tf.io.read_file(file_path)
#   img = decode_img(img)
#   return img, label

In [82]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
# labeled_ds_test = list_ds_test.map(process_path, num_parallel_calls=AUTOTUNE)

In [83]:
# def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  
#   if cache:
#     if isinstance(cache, str):
#       ds = ds.cache(cache)
#     else:
#       ds = ds.cache()

#   ds = ds.shuffle(buffer_size=shuffle_buffer_size)

#   # Repeat forever
#   ds = ds.repeat()

#   ds = ds.batch(BATCH_SIZE)

#   # `prefetch` lets the dataset fetch batches in the background while the model
#   # is training.
#   ds = ds.prefetch(buffer_size=AUTOTUNE)

#   return ds

In [84]:
# train_ds = prepare_for_training(labeled_ds)
# image_batch, label_batch = next(iter(train_ds))

In [85]:
# test_ds = labeled_ds_test.prefetch(buffer_size=AUTOTUNE)
# image_batch_test, label_batch_test = next(iter(test_ds))

In [75]:
#np.array([x[label][0] for label in label_batch]).reshape(-1,1)

In [39]:
# def show_batch(image_batch, label_batch):
#   plt.figure(figsize=(10,10))
#   for n in range(25):
#       ax = plt.subplot(5,5,n+1)
#       plt.imshow(image_batch[n])
#       plt.title(car_make_nums[label_batch[n]==1][0])
#       plt.axis('off')

# show_batch(image_batch.numpy(), label_batch.numpy())

In [25]:
# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(image_batch[i], cmap=plt.cm.binary)
#     plt.xlabel(np.array(car_makes['mnfr'])[label_batch[i]])
# plt.show()

In [25]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(196, activation='softmax'))

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 173056)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [28]:
cars_numerical = np.array(car_makes.index)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit((images_decoded, labels), epochs=10)